## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
%config Completer.use_jedi = False

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file.
weather_data_path = os.path.join("..","Weather_Database","WeatherPy_database.csv")
city_data_df = pd.read_csv(weather_data_path)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Spanaway,US,47.1040,-122.4346,73.45,64,90,0.00,overcast clouds
1,1,Vaini,TO,-21.2000,-175.2000,73.56,94,90,10.36,moderate rain
2,2,Tiksi,RU,71.6872,128.8694,48.65,64,95,3.27,overcast clouds
3,3,New Norfolk,AU,-42.7826,147.0587,54.91,72,100,1.99,light rain
4,4,Cherskiy,RU,68.7500,161.3000,44.85,76,100,14.99,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 73
What is the maximum temperature you would like for your trip? 77


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Spanaway,US,47.1040,-122.4346,73.45,64,90,0.00,overcast clouds
1,1,Vaini,TO,-21.2000,-175.2000,73.56,94,90,10.36,moderate rain
12,12,Hilo,US,19.7297,-155.0900,74.80,93,75,4.61,light rain
19,19,Atuona,PF,-9.8000,-139.0333,76.51,76,31,14.12,scattered clouds
45,45,Svetlyy,RU,54.6750,20.1347,75.25,54,44,10.78,scattered clouds
54,54,Richards Bay,ZA,-28.7830,32.0377,74.91,44,0,7.43,clear sky
69,69,Vostok,RU,46.4856,135.8833,74.17,76,92,4.97,overcast clouds
75,75,Sao Filipe,CV,14.8961,-24.4956,76.41,83,82,7.07,broken clouds
76,76,Aguimes,ES,27.9054,-15.4461,74.66,64,20,33.38,few clouds
85,85,Kahului,US,20.8947,-156.4700,76.26,81,1,14.97,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81 entries, 0 to 688
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              81 non-null     int64  
 1   City                 81 non-null     object 
 2   Country              81 non-null     object 
 3   Lat                  81 non-null     float64
 4   Lng                  81 non-null     float64
 5   Max Temp             81 non-null     float64
 6   Humidity             81 non-null     int64  
 7   Cloudiness           81 non-null     int64  
 8   Wind Speed           81 non-null     float64
 9   Current Description  81 non-null     object 
dtypes: float64(4), int64(3), object(3)
memory usage: 7.0+ KB


In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Spanaway,US,73.45,overcast clouds,47.1040,-122.4346,
1,Vaini,TO,73.56,moderate rain,-21.2000,-175.2000,
12,Hilo,US,74.80,light rain,19.7297,-155.0900,
19,Atuona,PF,76.51,scattered clouds,-9.8000,-139.0333,
45,Svetlyy,RU,75.25,scattered clouds,54.6750,20.1347,
54,Richards Bay,ZA,74.91,clear sky,-28.7830,32.0377,
69,Vostok,RU,74.17,overcast clouds,46.4856,135.8833,
75,Sao Filipe,CV,76.41,broken clouds,14.8961,-24.4956,
76,Aguimes,ES,74.66,few clouds,27.9054,-15.4461,
85,Kahului,US,76.26,clear sky,20.8947,-156.4700,


In [9]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81 entries, 0 to 688
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 81 non-null     object 
 1   Country              81 non-null     object 
 2   Max Temp             81 non-null     float64
 3   Current Description  81 non-null     object 
 4   Lat                  81 non-null     float64
 5   Lng                  81 non-null     float64
 6   Hotel Name           81 non-null     object 
dtypes: float64(3), object(4)
memory usage: 5.1+ KB


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print(f"Found: {index}")
    except (IndexError):
        print("Hotel not found... skipping.")
        

Found: 0
Found: 1
Found: 12
Found: 19
Found: 45
Found: 54
Hotel not found... skipping.
Found: 75
Found: 76
Found: 85
Found: 94
Found: 95
Found: 98
Hotel not found... skipping.
Found: 107
Found: 108
Found: 115
Found: 118
Found: 122
Found: 138
Found: 150
Found: 151
Found: 157
Found: 158
Found: 166
Found: 171
Found: 177
Found: 193
Found: 201
Found: 206
Found: 214
Found: 229
Found: 243
Hotel not found... skipping.
Found: 259
Found: 274
Found: 282
Found: 313
Found: 319
Found: 339
Found: 340
Found: 348
Found: 349
Found: 354
Found: 357
Found: 358
Found: 385
Found: 396
Found: 420
Hotel not found... skipping.
Hotel not found... skipping.
Found: 428
Found: 431
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Found: 463
Found: 467
Found: 483
Found: 485
Found: 488
Found: 496
Found: 504
Found: 507
Found: 529
Found: 533
Found: 548
Found: 561
Found: 569
Hotel not found... skipping.
Found: 580
Found: 603
Found: 625
Found: 634
Found: 652
Found: 670
Found: 674
Found

In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81 entries, 0 to 688
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 81 non-null     object 
 1   Country              81 non-null     object 
 2   Max Temp             81 non-null     float64
 3   Current Description  81 non-null     object 
 4   Lat                  81 non-null     float64
 5   Lng                  81 non-null     float64
 6   Hotel Name           81 non-null     object 
dtypes: float64(3), object(4)
memory usage: 7.1+ KB


In [12]:
# rows where there is no Hotel Name.
hotel_df[hotel_df['Hotel Name']==""].index


Int64Index([69, 106, 247, 425, 426, 435, 445, 448, 577], dtype='int64')

In [13]:
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81 entries, 0 to 688
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 81 non-null     object 
 1   Country              81 non-null     object 
 2   Max Temp             81 non-null     float64
 3   Current Description  81 non-null     object 
 4   Lat                  81 non-null     float64
 5   Lng                  81 non-null     float64
 6   Hotel Name           81 non-null     object 
dtypes: float64(3), object(4)
memory usage: 5.1+ KB


In [14]:
clean_hotel_df.drop(clean_hotel_df[hotel_df['Hotel Name']==""].index,axis=0,inplace=True)
clean_hotel_df


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Spanaway,US,73.45,overcast clouds,47.1040,-122.4346,Firwood RV
1,Vaini,TO,73.56,moderate rain,-21.2000,-175.2000,Keleti Beach Resort
12,Hilo,US,74.80,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
19,Atuona,PF,76.51,scattered clouds,-9.8000,-139.0333,Villa Enata
45,Svetlyy,RU,75.25,scattered clouds,54.6750,20.1347,"Gostinitsa-Kafe, Alena"
...,...,...,...,...,...,...,...
674,Nouakchott,MR,76.93,broken clouds,18.0858,-15.9785,HOTEL IMAN فندق إيمان
677,Zubtsov,RU,75.36,few clouds,56.1753,34.5894,Rogovo
686,Huntsville,US,75.33,mist,34.7304,-86.5861,Embassy Suites by Hilton Huntsville Hotel & Spa
687,Kuytun,RU,73.42,overcast clouds,54.3419,101.5036,Gostinitsa Tsentral'naya


In [15]:
clean_hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72 entries, 0 to 688
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 72 non-null     object 
 1   Country              72 non-null     object 
 2   Max Temp             72 non-null     float64
 3   Current Description  72 non-null     object 
 4   Lat                  72 non-null     float64
 5   Lng                  72 non-null     float64
 6   Hotel Name           72 non-null     object 
dtypes: float64(3), object(4)
memory usage: 4.5+ KB


In [16]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Spanaway,US,73.45,overcast clouds,47.1040,-122.4346,Firwood RV
1,Vaini,TO,73.56,moderate rain,-21.2000,-175.2000,Keleti Beach Resort
12,Hilo,US,74.80,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
19,Atuona,PF,76.51,scattered clouds,-9.8000,-139.0333,Villa Enata
45,Svetlyy,RU,75.25,scattered clouds,54.6750,20.1347,"Gostinitsa-Kafe, Alena"
...,...,...,...,...,...,...,...
674,Nouakchott,MR,76.93,broken clouds,18.0858,-15.9785,HOTEL IMAN فندق إيمان
677,Zubtsov,RU,75.36,few clouds,56.1753,34.5894,Rogovo
686,Huntsville,US,75.33,mist,34.7304,-86.5861,Embassy Suites by Hilton Huntsville Hotel & Spa
687,Kuytun,RU,73.42,overcast clouds,54.3419,101.5036,Gostinitsa Tsentral'naya


In [17]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
locations

,Lat,Lng
0,47.1040,-122.4346
1,-21.2000,-175.2000
12,19.7297,-155.0900
19,-9.8000,-139.0333
45,54.6750,20.1347
...,...,...
674,18.0858,-15.9785
677,56.1753,34.5894
686,34.7304,-86.5861
687,54.3419,101.5036


In [21]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))